In [17]:
import numpy as m 
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [35]:
data = pd.read_excel('3.2.3.xlsx').drop('Unnamed: 16',axis = 1)
data

,Cn_nF,fn,un_mV,en_mV,fn_1,un_1,fn_3,un_3,fn_1.1,un_1.1,D_x_1,x0_1,fn_3.1,un_3.1,D_x_3,x0_3
0,25.1,32300.0,500.0,96.5,31560,306,24339,174,34320.0,150.0,3.0,7.5,25405.0,87.0,4.2,10.4
1,33.2,28100.0,390.0,96.5,31662,330,24207,194,33750.0,195.0,2.9,7.5,24760.0,125.0,3.6,10.4
2,47.3,23590.0,284.0,96.4,31777,365,24121,210,33474.0,233.0,2.7,7.5,24384.0,167.0,3.1,10.4
3,57.4,21430.0,238.0,96.3,31836,384,24016,220,33164.0,292.0,2.4,7.5,24185.0,196.0,2.5,10.4
4,67.5,19690.0,204.0,96.3,31882,400,24009,230,32990.0,336.0,2.1,7.5,24063.0,220.0,2.0,10.4
5,82.7,17841.0,168.8,96.3,31920,435,23932,245,32817.0,387.0,1.8,7.5,23927.0,244.0,1.4,10.4
6,101.6,16073.0,139.6,96.2,32035,450,23840,260,32647.0,436.0,1.4,7.5,23726.0,275.0,0.9,10.4
7,NaN,NaN,NaN,NaN,32130,475,23745,275,32300.0,500.0,0.0,7.5,23590.0,284.0,0.0,10.4
8,NaN,NaN,NaN,NaN,32300,500,23590,284,32020.0,441.0,1.1,7.5,23347.0,259.0,1.3,10.4
9,NaN,NaN,NaN,NaN,32548,470,23425,270,31860.0,390.0,1.5,7.5,23175.0,225.0,2.0,10.4


In [19]:
scheme_char = data.iloc[:, 0:4].dropna()
R = 3.5 # в Омах
R_1 = 1008 # в Омах
def deviation_table(a):
    b = pd.DataFrame()
    b.loc['mean',a.name] = a.mean()
    b.loc['Random_deviation',a.name] = a.std()
    b.loc['Mean_square_deviation',a.name] = a.std()/a.size**0.5
    return b 
def surgery(k,*num,asc = False):
    d = k.copy()
    a = d[d.iloc[:,0]<=1]
    b = d[d.iloc[:,0]>1]
    if len(num) == 1:
        a.loc[a.shape[0]+1,a.keys()[1]] = num[0]
        b.loc[d.shape[0]+1,b.keys()[1]] = num[0]
    elif len(num) == 2:
        a.loc[a.shape[0]+1,a.keys()[1]] = num[0]
        b.loc[d.shape[0]+1,b.keys()[1]] = num[1]
    #print(a, b)
    a = a.set_index(a.keys()[1])
    b = b.set_index(b.keys()[1])
    a = a.sort_index()
    b = b.sort_index()
    a = a.interpolate(method='from_derivatives')
    b = b.interpolate(method='from_derivatives')
    b = b.sort_index(ascending=asc)
    #print(a, b)
    return [a, b]
def domega(d,num = round(0.5**0.5,3)):
    a, b = surgery(d,num)
    return b.loc[num]-a.loc[num]
def surgery_2(k):
    d = k.copy()
    a = d[d.iloc[:,0]<=1]
    b = d[d.iloc[:,0]>1]
    a.iloc[:,1] = (a.iloc[:,2]-a.iloc[:,1])/a.iloc[:,2] 
    b.iloc[:,1] = (b.iloc[:,2]+b.iloc[:,1])/b.iloc[:,2]
    #print(a, b)
    return pd.concat([b,a])

Дальше я создаю табличку по 10-му пункту, каждая строка отвечает за отдельный столбец
по формулам: $$L = \frac{1}{(2\pi f_n)^2C_{n}}$$
$$\rho = \sqrt{\frac{L}{C}} $$
$$Z_{рез} = \frac{UR_1}{E}$$
$$\sigma(\langle{X}\rangle)=\sqrt{\frac{1}{42}\Sigma(X_{i}-\langle{X}\rangle)^2}$$

In [53]:
scheme_char['L_mcHn'] = (10**15)/(scheme_char.Cn_nF*(2*m.pi*scheme_char.fn)**2)
scheme_char['p'] = (scheme_char.L_mcHn*1000/scheme_char.Cn_nF)**0.5
scheme_char['Z рез'] = R_1*scheme_char.un_mV/scheme_char.en_mV
scheme_char['Q'] = scheme_char['Z рез']/scheme_char.p
scheme_char['R_sum'] = scheme_char.p/scheme_char.Q
scheme_char['R_s'] = scheme_char.p*10**-3
scheme_char['R_l'] = scheme_char.R_sum-scheme_char.R_s-R 
#print(scheme_char)
scheme_dev = deviation_table(scheme_char.L_mcHn)
scheme_dev = scheme_dev.join(deviation_table(scheme_char.R_l))
#scheme_char
#scheme_dev.to_latex()
#sc = scheme_char.to_latex()

'\\begin{tabular}{lrr}\n\\toprule\n{} &      L\\_mcHn &       R\\_l \\\\\n\\midrule\nmean                  &  964.578494 &  3.203498 \\\\\nRandom\\_deviation      &    2.754303 &  0.280557 \\\\\nMean\\_square\\_deviation &    1.041029 &  0.106041 \\\\\n\\bottomrule\n\\end{tabular}\n'

Ниже два графика, один просто по данным, его можно на один рисунок, описание библиотеки: plotly.com/python

In [21]:
afc_data = data.iloc[:,4:8]
fig = make_subplots(rows=1,cols=2)
fig.add_trace(go.Scatter(x=afc_data.fn_1/1000, y=afc_data.un_1,
                         mode='lines+markers',
                         name='U(Fr) при положении 1',
                         line_shape='spline')
              ,row = 1,col = 1)
fig.add_trace(go.Scatter(x=afc_data.fn_3/1000, y=afc_data.un_3,
                         mode='lines+markers',
                         name='U(Fr) при положении 1',
                         line_shape='spline')
              ,row = 1,col = 2)

In [22]:
fig2 = make_subplots(rows=1,cols=2)
fig2.add_trace(go.Scatter(x = afc_data.fn_1/scheme_char.fn[0],
                          y=afc_data.un_1/scheme_char.un_mV[0],
                         mode='lines+markers',
                         name='U(Fr) при положении 1',
                         line_shape='spline')
              ,row = 1,col = 1)
fig2.add_trace(go.Scatter(x = afc_data.fn_3/scheme_char.fn[2],
                          y = afc_data.un_3/scheme_char.un_mV[2],
                         mode='lines+markers',
                         name='U(Fr) при положении 3',
                         line_shape='spline')
              ,row = 1,col = 2)

In [23]:
u_1 = afc_data.un_1/scheme_char.un_mV[0]
uf_1 = afc_data[['fn_1']]/scheme_char.fn[0]
uf_1 = uf_1.join(u_1)
delta = domega(uf_1)
uf_1 = pd.concat(surgery(uf_1,round(1/0.5**0.5)))
u_2 = afc_data.un_3/scheme_char.un_mV[2]
uf_2 = afc_data[['fn_3']]/scheme_char.fn[2]
uf_2 = uf_2.join(u_2)
delta_2 = domega(uf_2)
uf_2 = pd.concat(surgery(uf_2,round(1/0.5**0.5)))
Q_graph = pd.DataFrame({'1-е положение':1/delta,'3-е положение':1/delta_2})
Q_graph


C:\Users\vanan\Miniconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\vanan\Miniconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\vanan\Miniconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,1-е положение,3-е положение
fn_1,26.964175,NaN
fn_3,NaN,29.213098


Дальше я работаю на ФЧХ(PFC)

In [24]:
pfc_data = data.iloc[:,8:16].dropna()

pfc_data

,fn_1.1,un_1.1,D_x_1,x0_1,fn_3.1,un_3.1,D_x_3,x0_3
0,34320.0,150.0,3.0,7.5,25405.0,87.0,4.2,10.4
1,33750.0,195.0,2.9,7.5,24760.0,125.0,3.6,10.4
2,33474.0,233.0,2.7,7.5,24384.0,167.0,3.1,10.4
3,33164.0,292.0,2.4,7.5,24185.0,196.0,2.5,10.4
4,32990.0,336.0,2.1,7.5,24063.0,220.0,2.0,10.4
5,32817.0,387.0,1.8,7.5,23927.0,244.0,1.4,10.4
6,32647.0,436.0,1.4,7.5,23726.0,275.0,0.9,10.4
7,32300.0,500.0,0.0,7.5,23590.0,284.0,0.0,10.4
8,32020.0,441.0,1.1,7.5,23347.0,259.0,1.3,10.4
9,31860.0,390.0,1.5,7.5,23175.0,225.0,2.0,10.4


In [56]:
plot1_data = pfc_data.iloc[:,[0,2,3]]
plot1_data.loc[:,'fn_1.1'] = plot1_data['fn_1.1']/scheme_char.fn[0]
plot2_data = pfc_data.iloc[:,[4,6,7]]
plot2_data.loc[:,'fn_3.1'] = plot2_data['fn_3.1']/scheme_char.fn[2]

y_data_1 = surgery_2(plot1_data)
y_data_1 = y_data_1.drop('x0_1',axis=1)
y_data_1.loc[:,'ol'] = 1
y_data_1.loc[:,'type'] = '1-е положение'
y_data_2 = surgery_2(plot2_data)
y_data_2 = y_data_2.drop('x0_3',axis=1).rename(columns={"D_x_3": "D_x_1", "fn_3.1": "fn_1.1"})
y_data_2.loc[:,'ol'] = 1
y_data_2.loc[:,'type'] = '3-е положение'
squares = m.linalg.lstsq(y_data_1.iloc[:,[0,2]],y_data_1.iloc[:,1], rcond=-1)
squares2 = m.linalg.lstsq(y_data_2.iloc[:,[0,2]],y_data_2.iloc[:,1], rcond=-1)
y_data_1.loc[:,'deviation'] = y_data_1['fn_1.1']*float(squares[1])
y_data_2.loc[:,'deviation'] = y_data_2['fn_1.1']*float(squares2[1])
y_data = pd.concat([y_data_1,y_data_2])

C:\Users\vanan\Miniconda3\lib\site-packages\pandas\core\indexing.py:576: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



ImportError: Missing optional dependency 'xarray'.  Use pip or conda to install xarray.

In [42]:
squares = m.linalg.lstsq(y_data_1.iloc[:,[0,2]],y_data_1.iloc[:,1], rcond=-1)
squares2 = m.linalg.lstsq(y_data_2.iloc[:,[0,2]],y_data_2.iloc[:,1], rcond=-1)
print(squares2[1],squares[1])

[0.06229702] [0.0828719]


In [55]:
fig = px.scatter(y_data, x="fn_1.1",
                 y="D_x_1", facet_row='type',
                 trendline="ols",
                error_y=y_data['deviation'],
                 height=800, width=800)

In [49]:
y_data_1 = pd.concat(surgery(y_data_1.iloc[:,[0,1]],0.75,1.25,asc = True))
Q_1 = -1/(y_data_1.loc[0.75,'fn_1.1']-y_data_1.loc[1.25,'fn_1.1'])

C:\Users\vanan\Miniconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\vanan\Miniconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\vanan\Miniconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,fn_1.1,D_x_1,ol,type,deviation
0,1.062539,1.400000,1,1-е положение,0.088055
1,1.044892,1.386667,1,1-е положение,0.086592
2,1.036347,1.360000,1,1-е положение,0.085884
3,1.026749,1.320000,1,1-е положение,0.085089
4,1.021362,1.280000,1,1-е положение,0.084642
5,1.016006,1.240000,1,1-е положение,0.084198
6,1.010743,1.186667,1,1-е положение,0.083762
7,1.000000,1.000000,1,1-е положение,0.082872
8,0.991331,0.853333,1,1-е положение,0.082154
9,0.986378,0.800000,1,1-е положение,0.081743


In [70]:
y_data_1[['fn_1.1','D_x_1']].transpose().to_csv('matlab1.csv',sep=' ')
y_data_2[['fn_1.1','D_x_1']].transpose().to_csv('matlab2.csv',sep=' ')